# My first AutoGen project: Root cause analysis for pH value low - Task solved with provided function of time series analysis


AutoGen offers conversable agents powered by LLM, tool, or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation. Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

The use case is the following: a pH value low alert has been fired by some senser and notified a Chatbot (AssistantAgent) which in turn dispatch a UserProxyAgent to get the time series dataset, conduct a dynamic lower band calculation. If the dataset present datapoints with values lower than the lower band. If so, then calculate the estimated time the drop happened. Based on the results, the agent will suggest making a 'Rate Change'.
  
## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install `pyautogen`:
```bash
pip install pyautogen
```

In [27]:
# %pip install "pyautogen>=0.2.3"

## Set Open AI API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from a json file.

In [28]:
from typing import Literal

from pydantic import BaseModel, Field
from typing_extensions import Annotated

import autogen
from autogen.cache import Cache

config_list = autogen.config_list_from_json(
    "./OAI_CONFIG_LIST.json"  # All needed OpenAI info is in this file
)

The config list looks like the following:
```python
[
    {
        'model': 'gpt-3.5-turbo-16k',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2024-02-01',
        'tags': ['tool', '3.5-tool'],
    }
]
```

## Making Function Calls

In this example, we demonstrate function call execution with `AssistantAgent` and `UserProxyAgent`. With the new "function_call" feature, we define functions and specify the description of the function in the OpenAI config for the `AssistantAgent`. Then we register the functions in `UserProxyAgent`.


In [29]:
llm_config = {
    "config_list": config_list,
    "timeout": 120,
}

chatbot = autogen.AssistantAgent(
    name="chatbot",
    system_message="For pH value low tasks, only use the functions you have been provided with. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
)

# create a UserProxyAgent instance named "user_proxy" and configure it to not use docker
my_code_execution_config = dict(use_docker=False)
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config=my_code_execution_config,
)

import pandas as pd
import numpy as np

def interpolate_time(row1, row2, lower_band):
    time_diff = (row2['time'] - row1['time']).total_seconds()
    ph_diff = row2['pH'] - row1['pH']
    time_to_lower_band = (lower_band - row1['pH']) * time_diff / ph_diff
    exact_time = row1['time'] + pd.Timedelta(seconds=time_to_lower_band)
    return exact_time

def drop_time(std=1.0) -> str:
    # Sample data
    data = {
        'time': pd.date_range(start='2023-01-01', periods=10, freq='H'),
        'pH': [7.0, 6.8, 6.5, 6.2, 6.0, 5.8, 5.5, 5.3, 5.0, 4.8]
    }
    df = pd.DataFrame(data)
    
    # Calculate mean and standard deviation of pH values
    mean_ph = np.mean(df['pH'])
    std_ph = np.std(df['pH'])

    # Set lower band as 2 standard deviations below the mean
    lower_band = mean_ph - 1.0 * std
    print(f"Calculated lower band: {lower_band}")
    
    # Identify pH drops below the lower band value
    df['below_lower_band'] = df['pH'] < lower_band
    
    # Find the exact time when pH drops below the lower band value using interpolation
    drop_times = []
    for i in range(1, len(df)):
        if df.loc[i-1, 'pH'] >= lower_band and df.loc[i, 'pH'] < lower_band:
            exact_time = interpolate_time(df.loc[i-1], df.loc[i], lower_band)
            drop_times.append(exact_time)

    for time in drop_times:
        if time != None:
            print("Exact times when pH drops below the lower band value:")
            print(time)
            return str(time)
        else:
            print("No exact time found.")
            return None

@user_proxy.register_for_execution()
@chatbot.register_for_llm(description="pH Low ROA.")
def pHLowROA(
    DropTime: str = "A RATE CHANGE action is needed as the stimated time of pH drop below the dynamic lower band at 1 standard deviation is:"
) -> str:
    dropped_at = drop_time()
    if dropped_at == None:
        return "No exact time found and thus no action needed."
    else:
        return f"{DropTime} {dropped_at}"

The decorator `@chatbot.register_for_llm()` reads the annotated signature of the function `currency_calculator` and generates the following JSON schema used by OpenAI API to suggest calling the function. We can check the JSON schema generated as follows:

In [30]:
chatbot.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'pH Low ROA.',
   'name': 'pHLowROA',
   'parameters': {'type': 'object',
    'properties': {'DropTime': {'type': 'string',
      'default': 'A RATE CHANGE action is needed as the stimated time of pH drop below the dynamic lower band at 1 standard deviation is:',
      'description': 'DropTime'}},
    'required': []}}}]

The decorator `@user_proxy.register_for_execution()` maps the name of the function to be proposed by OpenAI API to the actual implementation. The function mapped is wrapped since we also automatically handle serialization of the output of function as follows:

- string are untouched, and

- objects of the Pydantic BaseModel type are serialized to JSON.

We can check the correctness of function map by using `._origin` property of the wrapped function as follows:

In [31]:
assert user_proxy.function_map["pHLowROA"]._origin == pHLowROA

Finally, we can use this function to accurately calculate exchange amounts:

In [32]:
with Cache.disk() as cache:
    # start the conversation
    res = user_proxy.initiate_chat(
        chatbot, message="'An pH value low' alert has been fired.", summary_method="reflection_with_llm", cache=cache
    )

user_proxy (to chatbot):

'An pH value low' alert has been fired.

--------------------------------------------------------------------------------
chatbot (to user_proxy):

***** Suggested tool call (call_MgqUAHL6FaMzqI4WAbNz7Ihm): pHLowROA *****
Arguments: 
{
  "DropTime": "A RATE CHANGE action is needed as the estimated time of pH drop below the dynamic lower band at 1 standard deviation is: 30 minutes."
}
*************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION pHLowROA...
Calculated lower band: 4.89
Exact times when pH drops below the lower band value:
2023-01-01 08:33:00
user_proxy (to chatbot):

user_proxy (to chatbot):

***** Response from calling tool (call_MgqUAHL6FaMzqI4WAbNz7Ihm) *****
A RATE CHANGE action is needed as the estimated time of pH drop below the dynamic lower band at 1 standard deviation is: 30 minutes. 2023-01-01 08:33:00
************

In [33]:
print("Chat summary:", res.summary)

Chat summary: A RATE CHANGE action is needed as the estimated time of pH drop below the dynamic lower band at 1 standard deviation is 30 minutes.
